# Lab 7 Extra - CIFAR10 with a deeper network and augmented images

## Imports, vars and loading data

In [1]:
from keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, InputLayer
from keras.optimizers import SGD, Adam, RMSprop
import matplotlib.pyplot as plt
import os
import tensorflow as tf

In [2]:
IMG_CHANNELS = 3
IMG_ROWS = 32
IMG_COLS = 32
BATCH_SIZE = 128
NB_EPOCH = 20
NB_CLASSES = 10
VERBOSE = 1
VALIDATION_SPLIT = 0.2
OPTIM = RMSprop()

In [3]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print('X_test shape:', X_test.shape)
print(X_test.shape[0], 'test samples')
# One-hot encode the target variables
Y_train = to_categorical(y_train, NB_CLASSES)
Y_test = to_categorical(y_test, NB_CLASSES)
# Normalize the images
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 13s 0us/step
X_train shape: (50000, 32, 32, 3)
50000 train samples
X_test shape: (10000, 32, 32, 3)
10000 test samples


## Model building & fitting

In [4]:
model = Sequential()
model.add(InputLayer(input_shape=(IMG_ROWS, IMG_COLS, IMG_CHANNELS)))
model.add(Conv2D(32, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(NB_CLASSES))
model.add(Activation('softmax'))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


In [5]:
model.compile(loss='categorical_crossentropy', optimizer = OPTIM, metrics = ['accuracy'])
model.fit(X_train, Y_train, batch_size = BATCH_SIZE,
          epochs = NB_EPOCH, validation_split = VALIDATION_SPLIT,
          verbose = VERBOSE)

Epoch 1/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 17s 31ms/step - accuracy: 0.2464 - loss: 2.0459 - val_accuracy: 0.4677 - val_loss: 1.4897
Epoch 2/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.4797 - loss: 1.4609 - val_accuracy: 0.5647 - val_loss: 1.2155
Epoch 3/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.5778 - loss: 1.2073 - val_accuracy: 0.6506 - val_loss: 0.9989
Epoch 4/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - accuracy: 0.6341 - loss: 1.0435 - val_accuracy: 0.6545 - val_loss: 0.9849
Epoch 5/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - accuracy: 0.6770 - loss: 0.9170 - val_accuracy: 0.6827 - val_loss: 0.9194
Epoch 6/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.7096 - loss: 0.8321 - val_accuracy: 0.7138 - val_loss: 0.8218
Epoch 7/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - accuracy: 0.7385 - loss: 0.7519 - val_accuracy: 0.7279 - val_loss: 0.7853
Epoch 8/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - accuracy: 0.7602 - loss: 0.6916 - val_acc

## Evaluating and saving

In [6]:
score = model.evaluate(X_test, Y_test, batch_size = BATCH_SIZE, verbose = VERBOSE)

print("Test score:", score[0])
print('Test accuracy:', score[1])

79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7499 - loss: 0.8518
Test score: 0.871123731136322
Test accuracy: 0.7490000128746033


In [ ]:
# Convert the model architecture to JSON format
model_json = model.to_json()
# Open a file in write mode ('w')
# and write the model architecture (JSON string) to the file
with open('Models/DeepCIFAR10.json', 'w') as file:
    file.write(model_json)
# Save the model's weights to a file.
# overwrite=True means to overwrite the file if it already exists
model.save_weights('Models/DeepCIFAR10.weights.h5', overwrite=True)

## Image data generation

In [8]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np

# Define the number of augmentations to create per image.
NUM_TO_AUGMENT = 5 # ! Under no cirumstances is this ever really possible in a 60k image dataset.

In [9]:
# Create an ImageDataGenerator object with specified augmentation parameters.
datagen = ImageDataGenerator(
    rotation_range=40, # Range of random rotation in degrees.
    width_shift_range=0.2, # Range for random horizontal shifts (fraction of total width).
    height_shift_range=0.2, # Range for random vertical shifts (fraction of total height).
    zoom_range=0.2, # Range for random zoom.
    horizontal_flip=True, # Randomly flip inputs horizontally.
    fill_mode='nearest' # Strategy for filling newly created pixels.
)

### Saving augmented files to a directory. Not recommended here.

## Model training on augmented data

### Model building (copied from above)

In [10]:
model = Sequential()
model.add(InputLayer(input_shape=(IMG_ROWS, IMG_COLS, IMG_CHANNELS)))
model.add(Conv2D(32, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(NB_CLASSES))
model.add(Activation('softmax'))

In [11]:
model.compile(loss = 'categorical_crossentropy', optimizer = OPTIM, metrics = ['accuracy'])

### Training

In [13]:
datagen.fit(X_train)

In [16]:
tf.executing_eagerly()

True

**Absolutely no idea why we're erroring here?** Eager execution is evidently on.
Something isn't right, and you may just have to wait for the lab sample to be posted instead.

In [14]:
history = model.fit(datagen.flow(X_train, Y_train, batch_size=BATCH_SIZE),
                    steps_per_epoch= int(np.ceil(X_train.shape[0] / BATCH_SIZE)),
                    epochs=NB_EPOCH, verbose=VERBOSE)

Epoch 1/20


NotImplementedError: numpy() is only available when eager execution is enabled.

In [ ]:
score = model.evaluate(X_test, Y_test,
batch_size=BATCH_SIZE, verbose=VERBOSE)
print("Test score:", score[0])
print('Test accuracy:', score[1])